# Notebook for initial analysis of the Tidal Data

## From this code we can start generating useful functions

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
import tide_utils as tide

In [ ]:
NeahBay = tide.load_Neah_Bay('../Data')
PortAngeles = tide.load_Port_Angeles('../Data')
PortTownsend = tide.load_Port_Townsend('../Data')

In [ ]:
del NeahBay, PortAngeles, PortTownsend

## Load the tidal csv data and put into a single dataframe and create a useful datetime index

In [6]:
def load_Neah_Bay(datadir):
    """
    Function to load the Neah Bay tidal station data from 2015 - 2016 
    and returns a dataframe and a Datetime Index object
    Datadir is the directory path to where the data is located
    """
    if not glob.glob(os.path.join(datadir, '*NeahBay.csv')):
        return None
    else:
        NeahBay_2014 = pd.read_csv(datadir + "2014_NeahBay.csv",parse_dates=['Date Time'])
        NeahBay_2015 = pd.read_csv(datadir + "2015_NeahBay.csv",parse_dates=['Date Time'])
        NeahBay_2016 = pd.read_csv(datadir + "2016_NeahBay.csv",parse_dates=['Date Time'])
        NeahBay = NeahBay_2014.append(NeahBay_2015)
        NeahBay = NeahBay.append(NeahBay_2016)
        NeahBay.rename(columns={'Date Time':'datetime'}, inplace=True)
        return NeahBay

In [ ]:
NeahBay = load_Neah_Bay('../Data/')

In [ ]:
NeahBay.head()

In [ ]:
def load_Port_Angeles(datadir):
    """
    Function to load the Port Angeles tidal station data from 2015 & 2016 
    Supply the directory to where the csv files with the data are saved
    Returns None if files are not located in specified directory
    """
    if not glob.glob(os.path.join(datadir, '*PortAngeles.csv')):
        return None
    else:
        # Load the Port Angeles tidal data and put into one dataframe
        PortAngeles_2014 = pd.read_csv(datadir + '/2014_PortAngeles.csv',parse_dates=['Date Time'])
        PortAngeles_2015 = pd.read_csv(datadir + '/2015_PortAngeles.csv',parse_dates=['Date Time'])
        PortAngeles_2016 = pd.read_csv(datadir + '/2016_PortAngeles.csv',parse_dates=['Date Time'])
        PortAngeles = PortAngeles_2014.append(PortAngeles_2015)
        PortAngeles = PortAngeles.append(PortAngeles_2016)
        PortAngeles.rename(columns={'Date Time':'datetime'}, inplace=True)
        return PortAngeles

In [ ]:
def load_Port_Townsend(datadir):
    """
    Function to load the Port Townsend tidal station data from 2015 & 2016 
    Supply the directory to where the csv files with the data are saved
    Returns None if files are not located in specified directory
    """
    if not glob.glob(os.path.join(datadir, '*PortTownsend.csv')):
        return None
    else:
        PortTownsend_2014 = pd.read_csv(datadir + '/2014_PortTownsend.csv',parse_dates=['Date Time'])
        PortTownsend_2015 = pd.read_csv(datadir + '/2015_PortTownsend.csv',parse_dates=['Date Time'])
        PortTownsend_2016 = pd.read_csv(datadir + '/2016_PortTownsend.csv',parse_dates=['Date Time'])
        PortTownsend = PortTownsend_2014.append(PortTownsend_2015)
        PortTownsend = PortTownsend.append(PortTownsend_2016)
        PortTownsend.rename(columns={'Date Time':'datetime'}, inplace=True)
        return PortTownsend

In [ ]:
def load_tide_data(datadir):
    """
    Upper level load function for the Tide Data.
    Datadir is the directory where the data .csv files are saved
    """
    NeahBay = load_Neah_Bay(datadir)
    PortAngeles = load_Port_Angeles(datadir)
    PortTownsend = load_Port_Townsend(datadir)
    return NeahBay, PortAngeles, PortTownsend
    

In [ ]:
import fnmatch
import os

def find_files(base, pattern):
    '''Return list of files matching pattern in base folder.'''
    return [n for n in fnmatch.filter(os.listdir(base), pattern) if
        os.path.isfile(os.path.join(base, n))]



In [ ]:
find_files('../Data/','*NB.csv')

In [ ]:
NeahBay, PortAngeles, PortTownsend = load_tide_data("../Data/")

In [ ]:
print(PortTownsend)

In [ ]:
def create_tide_dataset(NeahBay,PortAngeles,PortTownsend):
    """
    Function takes in the tidal station dataframes and returns
    an Xarray Dataset with the tidal station data
    """
    NB = xr.DataArray(NeahBay['Water Level'],dims='datetime')
    PA = xr.DataArray(PortAngeles['Water Level'],dims='datetime')
    PT = xr.DataArray(PortTownsend['Water Level'],dims='datetime')
    Tides = xr.Dataset({'NeahBay':NB, 'PortAngeles':PA, 'PortTownsend':PT})
    return Tides

In [ ]:
Tides = create_tide_dataset(NeahBay,PortAngeles,PortTownsend)

In [7]:
NeahBay = load_Neah_Bay('../Data/')

In [8]:
NeahBay.head()

,Unnamed: 0,datetime,Water Level,Sigma
0,0,2014-01-01 00:00:00,0.026,0.049
1,1,2014-01-01 00:06:00,-0.197,0.043
2,2,2014-01-01 00:12:00,-0.427,0.033
3,3,2014-01-01 00:18:00,-0.624,0.043
4,4,2014-01-01 00:24:00,-0.788,0.049


In [ ]:
Tides.NeahBay.sel(datetime=slice('2016-01-01','2016-01-02'))

In [ ]:
# Take a slice
NB = Tides.NeahBay.sel(datetime=slice('2015-01-01','2015-01-02'))
PA = Tides.PortAngeles.sel(datetime=slice('2015-01-01','2015-01-02'))
PT = Tides.PortTownsend.sel(datetime=slice('2015-01-01','2015-01-02'))

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
def plot_tide_data(dt):
    """
    This function plots the three tidal stations for the given
    time period along with a marker showing the time and elevation
    selected using the widget slider
    """
    fig, axes = plt.subplots(nrows=3)
    NB.plot(ax=axes[0])
    axes[0].scatter(x=NB.datetime.values[dt],y=NB.values[dt],color="red",s=100)
    axes[0].grid()

    PA.plot(ax=axes[1])
    axes[1].scatter(x=NB.datetime.values[dt],y=PA.values[dt],color="red",s=100)
    axes[1].grid()

    PT.plot(ax=axes[2])
    axes[2].scatter(x=NB.datetime.values[dt],y=PT.values[dt],color="red",s=100)
    axes[2].grid()
    
    plot_tidal_elevation(dt)

In [ ]:
slide = widgets.IntSlider(1,1,len(NB.datetime.values)-1)
interact(plot_tide_data,dt=slide)

In [ ]:
slide.value

In [ ]:
def plot_tidal_elevation(slide):
    try:
        # Create a figure with 3 rows & 1 column
        fig, axes = plt.subplots(nrows=1,ncols=1)
        # Get each station's tidal elevation based on the widget slider
        NBelev = NB.values[slide]
        PAelev = PA.values[slide]
        PTelev = PT.values[slide]
        # Create dummy x-values
        x=(1,2,3)
        y=(NBelev,PAelev,PTelev)
        # Create the figure with station labels
        plt.scatter(x,y,s=100,color="red",zorder=2)
        plt.plot(x,y,'b',zorder=1)
        plt.xticks(x,['Neah Bay','Port Angeles','Port Townsend'],rotation='vertical')
        plt.grid()
        plt.ylabel('Tidal Elevation (m)')
    except:
        return None

### Just do some plotting to make sure the data looks reasonable

In [ ]:
%matplotlib inline

In [ ]:
# Try taking the difference in the water level between the different stations
dif_Neah_Angeles = NeahBay[' Water Level'] - PortAngeles[' Water Level']

In [ ]:
NB, NB_dt = select_tide_data(NeahBay,NeahBay_datetime,['Jan 1 2016'],['Jan 31 2016 23:59:59'])
PA, PA_dt = select_tide_data(PortAngeles,PortAngeles_datetime,['Jan 1 2016'],['Jan 31 2016 23:59:59'])
PT, PT_dt = select_tide_data(PortTownsend,PortTownsend_datetime,['Jan 1 2016'],['Jan 31 2016 23:59:59'])

In [ ]:
dif_NB_PA['Water Level'] = NB[' Water Level'] - PA[' Water Level']

In [ ]:
dif_NB_PA['Date Time'] = NB['Date Time']

In [ ]:
NB[' Water Level'].groupby(NB_dt.date).mean().plot().grid()

In [ ]:
# Plot the daily mean of the tides data from Neah Bay stations
NeahBay[' Water Level'].groupby(NeahBay_datetime.date).mean().plot().grid()

In [ ]:
# Plot the daily mean of the tides data from Port Angeles stations
PortAngeles[' Water Level'].groupby(PortAngeles_datetime.date).mean().plot().grid()

In [ ]:
# Plot the daily mean of the tides data from Port Townsend stations
PortTownsend[' Water Level'].groupby(PortTownsend_datetime.date).mean().plot().grid()